## NBA Inter-City Travel Metrics Module
### First the NBA arena and aiport lat long as well as city, state, timezone and elevation details are extracted from
### https://wcupagis.maps.arcgis.com/home/webmap/viewer.html?layers=58b6e02819c04043b32e6237aa05c001
### Verified with https://www.stadium-maps.com/sports-maps/national-basketball-association.html\
###
### For geodesic distance calculations 
### !pip install geopy
###
### Authored by Murali Balasubramanian, DBA, Walsh College, MI, USA

In [2]:
import pandas as pd
from geopy.distance import geodesic
import pytz
from datetime import datetime

# -------------------------------
# STEP 1: Load Team Metadata
# -------------------------------
df = pd.read_excel("NBA_team_travel_data.xlsx")  # Columns: Team, Arena_lat, Arena_lng, city, state, airport_lat, airport_lng, timezone, Elevation (in ft)

# Normalize timezone names to pytz format
tz_map = {
    'Eastern': 'US/Eastern',
    'Central': 'US/Central',
    'Mountain': 'US/Mountain',
    'Pacific': 'US/Pacific'
}
df['pytz_tz'] = df['timezone'].map(tz_map)

# -------------------------------
# STEP 2: Arena-to-Airport Bus Time
# -------------------------------
def estimate_bus_time_arena_to_airport(arena_coords, airport_coords, city, base_speed_kmh=35):
    distance_km = geodesic(arena_coords, airport_coords).km
    congestion_map = {
        'Los Angeles': 1.5,
        'New York': 1.4,
        'Chicago': 1.3,
        'Atlanta': 1.2,
        'Salt Lake City': 1.1,
        'San Antonio': 1.0
    }
    congestion_factor = congestion_map.get(city, 1.2)
    bus_time_hr = (distance_km / base_speed_kmh) * congestion_factor
    return round(bus_time_hr, 2)

# -------------------------------
# STEP 3: Flight Time Estimation
# -------------------------------
def estimate_flight_time_km(distance_km, speed_kmh=600):
    return round(distance_km / speed_kmh, 2)

# -------------------------------
# STEP 4: Timezone Offset
# -------------------------------
def timezone_offset(from_tz, to_tz):
    now = datetime.utcnow()
    from_offset = pytz.timezone(from_tz).utcoffset(now)
    to_offset = pytz.timezone(to_tz).utcoffset(now)
    return round((to_offset - from_offset).total_seconds() / 3600, 2)

# -------------------------------
# STEP 5: Pairwise Travel Matrix
# -------------------------------
rows = []
for i, row_a in df.iterrows():
    for j, row_b in df.iterrows():
        if row_a['Team'] == row_b['Team']:
            continue

        team_from = row_a['Team']
        team_to = row_b['Team']

        # Arena and airport coordinates
        origin_airport = (row_a['airport_lat'], row_a['airport_lng'])
        dest_airport = (row_b['airport_lat'], row_b['airport_lng'])
        dest_arena = (row_b['Arena_lat'], row_b['Arena_lng'])

        # Flight distance and time
        flight_dist_km = geodesic(origin_airport, dest_airport).km
        flight_time_hr = estimate_flight_time_km(flight_dist_km)

        # Bus time (arena to airport at destination)
        bus_time_hr = estimate_bus_time_arena_to_airport(dest_arena, dest_airport, row_b['city'])

        # Timezone adjustment
        tz_offset_hr = timezone_offset(row_a['pytz_tz'], row_b['pytz_tz'])

        # Elevation delta
        elev_delta_ft = abs(row_a['Elevation (in ft)'] - row_b['Elevation (in ft)'])

        # Total travel time
        total_travel_hr = flight_time_hr + bus_time_hr

        # Non-rest time: travel + warmup + media post game commitments
        non_rest_hr = total_travel_hr + 1.0 + 1.5

        # Adjusted rest (timezone gain/loss)
        adjusted_rest_hr = -non_rest_hr + tz_offset_hr

        rows.append({
            'From': team_from,
            'To': team_to,
            'FlightDist_km': round(flight_dist_km, 1),
            'FlightTime_hr': flight_time_hr,
            'BusTime_hr': bus_time_hr,
            'TZ_Offset_hr': tz_offset_hr,
            'ElevationDelta_ft': elev_delta_ft,
            'TotalTravel_hr': round(total_travel_hr, 2),
            'NonRest_hr': round(non_rest_hr, 2),
            'AdjustedRest_hr': round(adjusted_rest_hr, 2)
        })

df_travel = pd.DataFrame(rows)

C:\Users\MuraliSAI\AppData\Local\Temp\ipykernel_40316\3839147179.py:47: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow()


In [4]:
df_travel.head()

,From,To,FlightDist_km,FlightTime_hr,BusTime_hr,TZ_Offset_hr,ElevationDelta_ft,TotalTravel_hr,NonRest_hr,AdjustedRest_hr
0,ATL,BOS,1521.5,2.54,0.15,0.0,1017,2.69,5.19,-5.19
1,ATL,BKN,1222.0,2.04,0.58,0.0,974,2.62,5.12,-5.12
2,ATL,CHA,364.1,0.61,0.33,0.0,302,0.94,3.44,-3.44
3,ATL,CHI,974.8,1.62,0.82,-1.0,430,2.44,4.94,-5.94
4,ATL,CLE,891.1,1.49,0.59,0.0,354,2.08,4.58,-4.58


In [5]:
df_travel.to_excel("NBA_travel_data.xlsx", index=False)

In [105]:
df_travel_transition = pd.read_excel('NBA_travel_data.xlsx')

In [106]:
df_travel_transition.head()

,From,To,FlightDist_km,FlightTime_hr,BusTime_hr,TZ_Offset_hr,ElevationDelta_ft,TotalTravel_hr,NonRest_hr,AdjustedRest_hr
0,ATL,ATL,0.0,0.00,0.50,0,0,0.50,2.50,-2.50
1,ATL,BOS,1521.5,2.54,0.15,0,1017,2.69,5.19,-5.19
2,ATL,BKN,1222.0,2.04,0.58,0,974,2.62,5.12,-5.12
3,ATL,CHA,364.1,0.61,0.33,0,302,0.94,3.44,-3.44
4,ATL,CHI,974.8,1.62,0.82,-1,430,2.44,4.94,-5.94


In [115]:
df_transition_perc = pd.read_excel('Transition_metric_travel_24_25.xlsx')

In [116]:
df_transition_perc.head(10)

,Date,Team,Home/Away,Opp,Venue,W/L,Team Pts,Opp Pts,Spread,Transition_percent_O,Transition_percent_D,Total_Transition_percent
0,2025-04-30,LAL,Home,MIN,LAL,L,96,103,-6.0,0.158,0.172,0.330
1,2025-04-27,LAL,Away,MIN,MIN,L,113,116,2.5,0.133,0.144,0.277
2,2025-04-25,LAL,Away,MIN,MIN,L,104,116,4.0,0.229,0.158,0.387
3,2025-04-22,LAL,Home,MIN,LAL,W,94,85,-5.5,0.128,0.115,0.243
4,2025-04-19,LAL,Home,MIN,LAL,L,95,117,-4.5,0.096,0.181,0.277
5,2025-04-13,LAL,Away,POR,POR,L,81,109,6.0,0.088,0.132,0.220
6,2025-04-11,LAL,Home,HOU,LAL,W,140,109,-14.0,0.179,0.167,0.346
7,2025-04-09,LAL,Away,DAL,DAL,W,112,97,-4.5,0.143,0.152,0.295
8,2025-04-08,LAL,Away,OKC,OKC,L,120,136,9.0,0.063,0.094,0.157
9,2025-04-06,LAL,Away,OKC,OKC,W,126,99,9.0,0.169,0.189,0.358


In [134]:
df_transition_perc.columns

Index(['Date', 'Team', 'Home/Away', 'Opp', 'Venue', 'W/L', 'Team Pts',
       'Opp Pts', 'Spread', 'Transition_percent_O', 'Transition_percent_D',
       'Total_Transition_percent'],
      dtype='object')

In [117]:
df_transition_perc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2640 entries, 0 to 2639
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      2640 non-null   object 
 1   Team                      2640 non-null   object 
 2   Home/Away                 2640 non-null   object 
 3   Opp                       2640 non-null   object 
 4   Venue                     2640 non-null   object 
 5   W/L                       2640 non-null   object 
 6   Team Pts                  2640 non-null   int64  
 7   Opp Pts                   2640 non-null   int64  
 8   Spread                    2640 non-null   float64
 9   Transition_percent_O      2640 non-null   float64
 10  Transition_percent_D      2640 non-null   float64
 11  Total_Transition_percent  2640 non-null   float64
dtypes: float64(4), int64(2), object(6)
memory usage: 247.6+ KB


In [118]:
# Convert to datetime type
df_transition_perc['Date'] = pd.to_datetime(df_transition_perc['Date'])

# First sort by Team to group teams together
# Then sort by Date within each team group
df_transition_perc = df_transition_perc.sort_values(['Team', 'Date'])


In [119]:
df_transition_perc.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2640 entries, 170 to 2558
Data columns (total 12 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Date                      2640 non-null   datetime64[ns]
 1   Team                      2640 non-null   object        
 2   Home/Away                 2640 non-null   object        
 3   Opp                       2640 non-null   object        
 4   Venue                     2640 non-null   object        
 5   W/L                       2640 non-null   object        
 6   Team Pts                  2640 non-null   int64         
 7   Opp Pts                   2640 non-null   int64         
 8   Spread                    2640 non-null   float64       
 9   Transition_percent_O      2640 non-null   float64       
 10  Transition_percent_D      2640 non-null   float64       
 11  Total_Transition_percent  2640 non-null   float64       
dtypes: datetime64[ns](1), f

In [120]:
df_transition_perc.head()

,Date,Team,Home/Away,Opp,Venue,W/L,Team Pts,Opp Pts,Spread,Transition_percent_O,Transition_percent_D,Total_Transition_percent
170,2024-10-23,ATL,Home,BKN,ATL,W,120,116,-7.0,0.206,0.144,0.350
169,2024-10-25,ATL,Home,CHA,ATL,W,125,120,-5.5,0.252,0.136,0.388
168,2024-10-27,ATL,Away,OKC,OKC,L,104,128,10.5,0.094,0.175,0.269
167,2024-10-28,ATL,Home,WAS,ATL,L,119,121,-8.0,0.183,0.125,0.308
166,2024-10-30,ATL,Away,WAS,WAS,L,120,133,-5.5,0.144,0.171,0.315


In [123]:
df_transition_perc.reset_index(inplace=True)

In [124]:
df_transition_perc.head()

,index,Date,Team,Home/Away,Opp,Venue,W/L,Team Pts,Opp Pts,Spread,Transition_percent_O,Transition_percent_D,Total_Transition_percent
0,170,2024-10-23,ATL,Home,BKN,ATL,W,120,116,-7.0,0.206,0.144,0.350
1,169,2024-10-25,ATL,Home,CHA,ATL,W,125,120,-5.5,0.252,0.136,0.388
2,168,2024-10-27,ATL,Away,OKC,OKC,L,104,128,10.5,0.094,0.175,0.269
3,167,2024-10-28,ATL,Home,WAS,ATL,L,119,121,-8.0,0.183,0.125,0.308
4,166,2024-10-30,ATL,Away,WAS,WAS,L,120,133,-5.5,0.144,0.171,0.315


In [131]:
df_transition_perc.drop('index', axis =1, inplace=True )

In [132]:
df_transition_perc.head()

,Date,Team,Home/Away,Opp,Venue,W/L,Team Pts,Opp Pts,Spread,Transition_percent_O,Transition_percent_D,Total_Transition_percent
0,2024-10-23,ATL,Home,BKN,ATL,W,120,116,-7.0,0.206,0.144,0.350
1,2024-10-25,ATL,Home,CHA,ATL,W,125,120,-5.5,0.252,0.136,0.388
2,2024-10-27,ATL,Away,OKC,OKC,L,104,128,10.5,0.094,0.175,0.269
3,2024-10-28,ATL,Home,WAS,ATL,L,119,121,-8.0,0.183,0.125,0.308
4,2024-10-30,ATL,Away,WAS,WAS,L,120,133,-5.5,0.144,0.171,0.315


In [135]:
import pandas as pd

def transfer_travel_data(df_transition_perc, df_travel_transition):
    """
    Transfer travel-related data from df_travel_transition to df_transition_perc
    based on venue transitions and home/away status.
    """
    # Create a dictionary to store the transferred data for each team
    result_data = []
    
    # Process each team separately
    for team in df_transition_perc['Team'].unique():
        # Get all rows for current team
        team_rows = df_transition_perc[df_transition_perc['Team'] == team].reset_index(drop=True)
        
        # Process first game of the team
        first_row = team_rows.iloc[0]
        if first_row['Home/Away'] == 'Home':
            # Home game - Team plays at home venue
            matching_row = df_travel_transition[
                (df_travel_transition['From'] == team) & 
                (df_travel_transition['To'] == team)
            ].iloc[0]
        else:
            # Away game - Team travels to opponent's venue
            matching_row = df_travel_transition[
                (df_travel_transition['From'] == team) & 
                (df_travel_transition['To'] == first_row['Opp'])
            ].iloc[0]
            
        # Add travel data to first row
        travel_columns = [
            'FlightDist_km', 'FlightTime_hr', 'BusTime_hr', 
            'TZ_Offset_hr', 'ElevationDelta_ft', 'TotalTravel_hr',
            'NonRest_hr', 'AdjustedRest_hr'
        ]
        result_data.append({
            **first_row.to_dict(),
            **{col: matching_row[col] for col in travel_columns}
        })
        
        # Process remaining games for the team
        for i in range(1, len(team_rows)):
            current_row = team_rows.iloc[i]
            previous_venue = team_rows.iloc[i-1]['Venue']
            current_venue = current_row['Venue']
            
            # Find matching transition data
            matching_row = df_travel_transition[
                (df_travel_transition['From'] == previous_venue) & 
                (df_travel_transition['To'] == current_venue)
            ].iloc[0]
            
            # Add travel data to current row
            result_data.append({
                **current_row.to_dict(),
                **{col: matching_row[col] for col in travel_columns}
            })
    
    # Convert results to DataFrame and return
    return pd.DataFrame(result_data)

In [136]:
result_df = transfer_travel_data(df_transition_perc, df_travel_transition)

In [137]:
result_df.head()

,Date,Team,Home/Away,Opp,Venue,W/L,Team Pts,Opp Pts,Spread,Transition_percent_O,Transition_percent_D,Total_Transition_percent,FlightDist_km,FlightTime_hr,BusTime_hr,TZ_Offset_hr,ElevationDelta_ft,TotalTravel_hr,NonRest_hr,AdjustedRest_hr
0,2024-10-23,ATL,Home,BKN,ATL,W,120,116,-7.0,0.206,0.144,0.350,0.0,0.00,0.50,0,0,0.50,2.50,-2.50
1,2024-10-25,ATL,Home,CHA,ATL,W,125,120,-5.5,0.252,0.136,0.388,0.0,0.00,0.50,0,0,0.50,2.50,-2.50
2,2024-10-27,ATL,Away,OKC,OKC,L,104,128,10.5,0.094,0.175,0.269,1224.1,2.04,0.36,-1,176,2.40,4.90,-5.90
3,2024-10-28,ATL,Home,WAS,ATL,L,119,121,-8.0,0.183,0.125,0.308,1224.1,2.04,0.45,1,176,2.49,4.99,-3.99
4,2024-10-30,ATL,Away,WAS,WAS,L,120,133,-5.5,0.144,0.171,0.315,884.7,1.47,0.03,0,977,1.50,4.00,-4.00


In [138]:
result_df.to_excel('travel_transition_data_2024_25.xlsx', index=False)
